<h1 align="center"> Capstone Project-Battle of Neighborhood </h1>

###  Is Ahmedabad City similar to Toronto city?

In this project, we will study in details the area and doing classification using Foursquare data and machine learning segmentation and clustering. The aim of this project is to segment areas of Ahmedabad and Toronto based on the most common places captured from Foursquare.

Using segmentation and clustering, we hope we can determine:
   >	The similarity or dissimilarity of both cities  
   >	Classification of area located inside the city whether it is residential, tourism places, or others  
   >	Find an optimal location for establish a business  
   >	Restaurant food popularity   
   >	Identify optimal place for Bus station, Railway station or Airport   
  

## Import Libraries

In [308]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library


from bs4 import BeautifulSoup

print('Libraries imported.')

Libraries imported.


# Part-1:Load Data

### Get Ahmedabad city data

In [311]:
#create dataframe
df_ah = pd.read_csv('ahmedabad.csv',delimiter=',', header = None)
df_ah.columns = ['Neighborhood','Latitude','Longitude']

df_ah.head(10)

,Neighborhood,Latitude,Longitude
0,Asarwa,23.044980,72.607700
1,Ashram Road,23.006986,72.557490
2,Astodia,23.016830,72.590975
3,Bapunagar,23.032660,72.629494
4,Behrampura,23.003589,72.583840
5,Bodakdev,23.044371,72.517921
6,Bopal,23.030050,72.464943
7,CG Road,23.039084,72.562786
8,Chandkheda,23.112650,72.583618
9,Chandlodia,23.093990,72.546181


In [313]:
print("Number of rows in df_to dataframe: ",df_ah.shape)

Number of rows in df_to dataframe:  (39, 3)


### Get Toronto city data

In [314]:
url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

In [315]:
soup = BeautifulSoup(url,'lxml')

In [316]:
# print(soup.prettify())

In [317]:
#find table and and it cells

table = soup.find('table')

row_info = table.find_all('td')

In [318]:
print(type(row_info))

<class 'bs4.element.ResultSet'>


In [319]:
#columns for dataframe
postcode = []
borough = []
neighbourhood = []

for i in range(0, len(row_info), 3):
    postcode.append(row_info[i].text.strip())
    borough.append(row_info[i+1].text.strip())
    neighbourhood.append(row_info[i+2].text.strip())

In [320]:
#create dataframe
df_to = pd.DataFrame(data=[postcode, borough, neighbourhood]).transpose()
df_to.columns = ['Postcode', 'Borough', 'Neighborhood']

df_to.head()

,Postcode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


## Ignore cells with a borough that is Not assigned

In [321]:
a = df_to.index[df_to['Borough'] == 'Not assigned'].tolist()
print("Dropped Index is:",a)

df_to.drop(df_to.index[a],inplace=True)

df_to.head()

Dropped Index is: [0, 1, 9, 13, 20, 21, 30, 36, 37, 45, 46, 50, 51, 52, 54, 55, 59, 60, 61, 73, 74, 75, 88, 89, 90, 104, 105, 106, 120, 121, 136, 137, 148, 149, 155, 161, 162, 167, 175, 181, 182, 188, 189, 190, 194, 195, 201, 202, 203, 204, 209, 210, 223, 224, 238, 239, 242, 243, 248, 249, 254, 255, 259, 260, 261, 262, 264, 265, 275, 276, 277, 278, 279, 280, 281, 282, 288]


,Postcode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


## Combine rows which have same Borough and will be combined into one row with the neighborhoods separated with a comma,

In [322]:
df_to = df_to.groupby(['Postcode', 'Borough'])['Neighborhood'].apply(', '.join).reset_index()
df_to.columns = ['Postcode', 'Borough', 'Neighborhood']
df_to.head(10)

,Postcode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


## Find a row who have a borough but a Not assigned neighborhood,After that we will replace it with default Queen's Park.

In [323]:
df_to.index[df_to['Neighborhood'] == 'Not assigned'].tolist()

[85]

In [324]:
df_to['Neighborhood'].replace('Not assigned', "Queen's Park", inplace=True)

df_to.head(5)

,Postcode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [325]:
df_to.shape

(103, 3)

In [326]:
print("Number of rows in df_to dataframe: ",df_to.shape)

Number of rows in df_to dataframe:  (103, 3)


In [327]:
toronto_geo_data = pd.read_csv("http://cocl.us/Geospatial_data")
print("data loaded.")
toronto_geo_data.head()

data loaded.


,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [328]:
toronto_geo_data.columns = ['Postcode', 'Latitude', 'Longitude']
toronto_geo_data.head()

,Postcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


### Merge two dataset 

In [329]:
df_neighbors = pd.merge(df_to,toronto_geo_data,on=['Postcode'],how='inner')
df_neighbors.head(15)

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


In [330]:
neighborhoods = df_neighbors

##### here,for information simpicity with foursquare api we take oronto Neighbourhoods-boroughs that contain the word Toronto

### Toronto Neighbourhoods-boroughs that contain the word Toronto 

In [331]:
df_toronto = neighborhoods[neighborhoods['Borough'].str.contains('Toronto')].reset_index(drop=True) #necessary for new data
df_toronto.head()

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


# Part-3 Explore the neighborhoods in Ahmedabad and toronto.

## Both cities map and neighborhoods:

##### get ahmedabad geolocation

In [332]:
address = 'Ahmedabad, India'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of the City of Ahmedabad are {}, {}.'.format(latitude, longitude))

/Users/apple/Desktop/Python/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: DeprecationWarning: Using Nominatim with the default "geopy/1.18.1" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  This is separate from the ipykernel package so we can avoid doing imports until


The geograpical coordinate of the City of Ahmedabad are 23.0216238, 72.5797068.


##### get toronto geolocation

In [335]:
address1 = 'Toronto, Canada'

geolocator1 = Nominatim()
location1 = geolocator1.geocode(address1)
latitude1 = location1.latitude
longitude1 = location1.longitude
print('The geograpical coordinate of the City of Toronto are {}, {}.'.format(latitude1, longitude1))

/Users/apple/Desktop/Python/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: DeprecationWarning: Using Nominatim with the default "geopy/1.18.1" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  This is separate from the ipykernel package so we can avoid doing imports until


The geograpical coordinate of the City of Toronto are 43.653963, -79.387207.


### Map of ahmedabad neighbourhoods only:

In [336]:
map_ahmedabad_neigh = folium.Map(location=[latitude,longitude], zoom_start=11)

for lat, lng, neighborhood in zip(df_ah['Latitude'],df_ah['Longitude'],df_ah['Neighborhood']):
    label = '{}, {}, {}'.format(neighborhood, borough,postcode)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_ahmedabad_neigh) 

In [337]:
map_ahmedabad_neigh

###  Map of torrnto neighbourhoods only:

In [339]:
map_toronto_neigh = folium.Map(location=[latitude1,longitude1], zoom_start=11)

for lat, lng, borough, neighborhood,postcode in zip(df_toronto['Latitude'],df_toronto['Longitude'],df_toronto['Borough'],df_toronto['Neighborhood'],df_toronto['Postcode']):
    label = '{}, {}, {}'.format(neighborhood, borough,postcode)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_toronto_neigh) 
    


In [340]:
map_toronto_neigh

### Define Foursquare Credentials 

In [341]:
CLIENT_ID = 'F5B4QDBCYPQ1ZMSJDVTAYEECCTMBRQNZTE3MQZOJ15WTEYBT' # your Foursquare ID
CLIENT_SECRET = 'W514NY3S2O2WC5DXF0AR5ABFIKI13T2XEZFRR4MLQSKMMTVH' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: F5B4QDBCYPQ1ZMSJDVTAYEECCTMBRQNZTE3MQZOJ15WTEYBT
CLIENT_SECRET:W514NY3S2O2WC5DXF0AR5ABFIKI13T2XEZFRR4MLQSKMMTVH


# part-2:  (1)explore the Ahmedabad City  neighborhoods.

### Let's explore the first neighborhood in ahmedabad dataframe.

In [342]:
df_ah.loc[0, 'Neighborhood']

'Asarwa'

###  Get info about neighbourhood and also latitude and longitude

In [343]:
first_ah_neighborhood_latitude =  df_ah.loc[0,'Latitude']# neighborhood latitude value
first_ah_neighborhood_longitude = df_ah.loc[0,'Longitude'] # neighborhood longitude value

first_ah_neighborhood_name = df_ah.loc[0,'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(first_ah_neighborhood_name,
                                                               first_ah_neighborhood_latitude,first_ah_neighborhood_longitude))

Latitude and longitude values of Asarwa are 23.04498, 72.6077.


## Now, let's get the top 100 venues that are in venues within a radius of 500 meters.¶

In [344]:
Limit=100
radius = 500 
url = "https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
    CLIENT_ID,CLIENT_SECRET,VERSION,first_ah_neighbourhood_latitude,first_ah_neighbourhood_longitude,radius,Limit)

url

'https://api.foursquare.com/v2/venues/explore?&client_id=F5B4QDBCYPQ1ZMSJDVTAYEECCTMBRQNZTE3MQZOJ15WTEYBT&client_secret=W514NY3S2O2WC5DXF0AR5ABFIKI13T2XEZFRR4MLQSKMMTVH&v=20180605&ll=23.04498,72.6077&radius=500&limit=100'

In [345]:
#get the results
import requests

results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5c39c4a76a6071298e929599'},
 'response': {'warning': {'text': "There aren't a lot of results near you. Try something more general, reset your filters, or expand the search area."},
  'headerLocation': 'Ahmedabad',
  'headerFullLocation': 'Ahmedabad',
  'headerLocationGranularity': 'city',
  'totalResults': 2,
  'suggestedBounds': {'ne': {'lat': 23.049480004500005,
    'lng': 72.61258112702107},
   'sw': {'lat': 23.040479995499993, 'lng': 72.60281887297892}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '52b66a1e11d2f696eede67ac',
       'name': 'Asarwa railway yard',
       'location': {'lat': 23.041214,
        'lng': 72.606364,
        'labeledLatLngs': [{'label': 'display',
          'lat': 23.041214,
          'lng': 72.606364}]

In [346]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [347]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']

nearby_venues =nearby_venues.loc[:, filtered_columns]


# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1) #return each filter column name

print(nearby_venues.columns)

# clean columns
nearby_venues.columns = [col.split('.')[-1] for col in nearby_venues.columns]

nearby_venues.head()

Index(['venue.name', 'venue.categories', 'venue.location.lat',
       'venue.location.lng'],
      dtype='object')


,name,categories,lat,lng
0,Asarwa railway yard,Tea Room,23.041214,72.606364
1,Rudra Softech,Business Service,23.045616,72.612162


# And how many venues around Asarwa were  returned by Foursquare

In [348]:
nearby_venues.shape

(2, 4)

In [349]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

2 venues were returned by Foursquare.


### Let's create a function to repeat the same process to all the neighborhoods in Ahmedabad

In [350]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]

    for name, lat, lng in zip(names, latitudes, longitudes):
        #print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID,
            CLIENT_SECRET,
            VERSION,
            lat, 
            lng, 
            radius, 
            Limit)
        
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        
        # return only relevant information for each nearby venue,here name,lat,lag are neighbourhood features
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])
        
    
    nearby_venues = pd.DataFrame([item for venue_list in venues_list
                                          for item in venue_list])
    
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
     
    return(nearby_venues)


### Get the venues

In [351]:
ahmedabad_venues = getNearbyVenues(names=df_ah['Neighborhood'],
                                   latitudes=df_ah['Latitude'],
                                   longitudes=df_ah['Longitude'])

### Let's check the size of the resulting dataframe

In [352]:
print(ahmedabad_venues.shape)
ahmedabad_venues.head()

(260, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Asarwa,23.044980,72.60770,Asarwa railway yard,23.041214,72.606364,Tea Room
1,Asarwa,23.044980,72.60770,Rudra Softech,23.045616,72.612162,Business Service
2,Ashram Road,23.006986,72.55749,Domino's Pizza,23.006000,72.556000,Pizza Place
3,Ashram Road,23.006986,72.55749,cafe coffee day,23.008431,72.559805,Coffee Shop
4,Ashram Road,23.006986,72.55749,Maharaj Pan,23.005784,72.556983,Smoke Shop


In [353]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [354]:
ahmedabad_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Asarwa,2,2,2,2,2,2
Ashram Road,8,8,8,8,8,8
Astodia,4,4,4,4,4,4
Bapunagar,4,4,4,4,4,4
Behrampura,3,3,3,3,3,3
Bodakdev,9,9,9,9,9,9
Bopal,3,3,3,3,3,3
CG Road,15,15,15,15,15,15
Chandkheda,3,3,3,3,3,3


### Check about unique category

In [355]:
print('There are {} uniques categories.'.format(len(ahmedabad_venues['Venue Category'].unique())))

There are 84 uniques categories.


## Analyze Each Ahmedabad Neighborhood

In [356]:
# one hot encodinga
ahmedabad_onehot = pd.get_dummies(ahmedabad_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
ahmedabad_onehot['Neighborhood'] = ahmedabad_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [ahmedabad_onehot.columns[-1]] + list(ahmedabad_onehot.columns[:-1])

ahmedabad_onehot = ahmedabad_onehot[fixed_columns]

ahmedabad_onehot.head()

,Neighborhood,ATM,American Restaurant,Arcade,Art Museum,Arts & Crafts Store,Asian Restaurant,BBQ Joint,Bakery,Baseball Field,Bistro,Bookstore,Boutique,Breakfast Spot,Bubble Tea Shop,Bus Station,Bus Stop,Business Service,Café,Campground,Chinese Restaurant,City,Clothing Store,Coffee Shop,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Cricket Ground,Department Store,Dessert Shop,Diner,Donut Shop,Electronics Store,Fast Food Restaurant,Flea Market,Food,Food Court,Food Truck,Fried Chicken Joint,Furniture / Home Store,Garden,Garden Center,Gujarati Restaurant,Historic Site,History Museum,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Italian Restaurant,Jewelry Store,Juice Bar,Lake,Light Rail Station,Market,Men's Store,Mexican Restaurant,Mobile Phone Shop,Movie Theater,Moving Target,Multiplex,Noodle House,Park,Performing Arts Venue,Pharmacy,Pizza Place,Racetrack,Restaurant,Sandwich Place,Shopping Mall,Smoke Shop,Snack Place,Spa,Speakeasy,Tea Room,Tennis Court,Toy / Game Store,Track Stadium,Trail,Vegetarian / Vegan Restaurant,Wings Joint,Women's Store
0,Asarwa,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
1,Asarwa,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Ashram Road,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Ashram Road,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Ashram Road,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0


In [357]:
ahmedabad_onehot.shape

(260, 85)

### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category



In [358]:
ahmedabad_grouped = ahmedabad_onehot.groupby('Neighborhood').mean().reset_index()
ahmedabad_grouped

,Neighborhood,ATM,American Restaurant,Arcade,Art Museum,Arts & Crafts Store,Asian Restaurant,BBQ Joint,Bakery,Baseball Field,Bistro,Bookstore,Boutique,Breakfast Spot,Bubble Tea Shop,Bus Station,Bus Stop,Business Service,Café,Campground,Chinese Restaurant,City,Clothing Store,Coffee Shop,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Cricket Ground,Department Store,Dessert Shop,Diner,Donut Shop,Electronics Store,Fast Food Restaurant,Flea Market,Food,Food Court,Food Truck,Fried Chicken Joint,Furniture / Home Store,Garden,Garden Center,Gujarati Restaurant,Historic Site,History Museum,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Italian Restaurant,Jewelry Store,Juice Bar,Lake,Light Rail Station,Market,Men's Store,Mexican Restaurant,Mobile Phone Shop,Movie Theater,Moving Target,Multiplex,Noodle House,Park,Performing Arts Venue,Pharmacy,Pizza Place,Racetrack,Restaurant,Sandwich Place,Shopping Mall,Smoke Shop,Snack Place,Spa,Speakeasy,Tea Room,Tennis Court,Toy / Game Store,Track Stadium,Trail,Vegetarian / Vegan Restaurant,Wings Joint,Women's Store
0,Asarwa,0.00,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.500000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.00,0.0,0.000000,0.000000,0.000000,0.000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000,0.000000,0.000000,0.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
1,Ashram Road,0.00,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.125000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.125,0.000000,0.000000,0.000000,0.000000,0.000000,0.125,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.250000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.00,0.0,0.000000,0.000000,0.000000,0.000,0.00,0.125000,0.000000,0.000000,0.000000,0.000000,0.125,0.125,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
2,Astodia,0.00,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.750000,0.00,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.00,0.0,0.000000,0.000000,0.000000,0.000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000,0.000000,0.000000,0.250000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
3,Bapunagar,0.25,0.000000,0.000000,0.00,0.250,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.250000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.25,0.0,0.000000,0.000000,0.000000,0.000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
4,Behrampura,0.00,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.000000,0.333333,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.333333,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000

In [359]:
ahmedabad_grouped.shape

(39, 85)

### Let's print each neighborhood along with the top 5 most common venues

In [360]:
num_top_venues = 5

for hood in ahmedabad_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = ahmedabad_grouped[ahmedabad_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Asarwa----
              venue  freq
0  Business Service   0.5
1          Tea Room   0.5
2               ATM   0.0
3              Lake   0.0
4     Moving Target   0.0


----Ashram Road----
               venue  freq
0  Indian Restaurant  0.25
1        Snack Place  0.12
2        Flea Market  0.12
3        Pizza Place  0.12
4   Department Store  0.12


----Astodia----
           venue  freq
0    Bus Station  0.75
1       Tea Room  0.25
2            ATM  0.00
3           Lake  0.00
4  Moving Target  0.00


----Bapunagar----
                 venue  freq
0                  ATM  0.25
1        Movie Theater  0.25
2  Arts & Crafts Store  0.25
3    Indian Restaurant  0.25
4                 Lake  0.00


----Behrampura----
              venue  freq
0       Wings Joint  0.33
1    Baseball Field  0.33
2  Business Service  0.33
3               ATM  0.00
4              Lake  0.00


----Bodakdev----
               venue  freq
0        Coffee Shop  0.33
1  Indian Restaurant  0.11
2               Ca

### Function for sorting values

In [361]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

### create the new dataframe and display the top 10 venues for each neighborhood.



In [362]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
ah_neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
ah_neighborhoods_venues_sorted['Neighborhood'] = ahmedabad_grouped['Neighborhood']

for ind in np.arange(ahmedabad_grouped.shape[0]):
     ah_neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(ahmedabad_grouped.iloc[ind, :], num_top_venues)



### SO, here are the Ahmedabad Neighborhood with 10 venues for each neighborhood.

In [363]:
ah_neighborhoods_venues_sorted.head(10)

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Asarwa,Business Service,Tea Room,Women's Store,Department Store,Coffee Shop,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop
1,Ashram Road,Indian Restaurant,Smoke Shop,Department Store,Coffee Shop,Flea Market,Pizza Place,Snack Place,Comfort Food Restaurant,Concert Hall,Construction & Landscaping
2,Astodia,Bus Station,Tea Room,Women's Store,Department Store,Coffee Shop,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop
3,Bapunagar,ATM,Arts & Crafts Store,Indian Restaurant,Movie Theater,Food,Food Court,Coffee Shop,Comfort Food Restaurant,Concert Hall,Construction & Landscaping
4,Behrampura,Wings Joint,Business Service,Baseball Field,Dessert Shop,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Cricket Ground
5,Bodakdev,Coffee Shop,Breakfast Spot,Café,Hotel,Tea Room,Indian Restaurant,Bubble Tea Shop,Women's Store,Concert Hall,Construction & Landscaping
6,Bopal,Mobile Phone Shop,Park,Fast Food Restaurant,Women's Store,Cricket Ground,Coffee Shop,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store
7,CG Road,Indian Restaurant,Pizza Place,Cricket Ground,Food,Fast Food Restaurant,Café,Ice Cream Shop,Clothing Store,Jewelry Store,Juice Bar
8,Chandkheda,Shopping Mall,Breakfast Spot,Indian Restaurant,Women's Store,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Cricket Ground
9,Chandlodia,Snack Place,Women's Store,Coffee Shop,Indian Restaurant,Dessert Shop,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop


# part-2: (2) Explore the Toronto City neighborhoods.

#### Explore first neighborhood

In [364]:
df_toronto.loc[0, 'Neighborhood']

'The Beaches'

###  Get info about neighbourhood and also latitude and longitude

In [365]:
first_to_neighborhood_latitude =  df_toronto.loc[0,'Latitude']# neighborhood latitude value
first_to_neighborhood_longitude = df_toronto.loc[0,'Longitude'] # neighborhood longitude value

first_to_neighborhood_name = df_toronto.loc[0,'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(first_to_neighborhood_name,
                                                               first_to_neighborhood_latitude,first_to_neighborhood_longitude))

Latitude and longitude values of The Beaches are 43.67635739999999, -79.2930312.


## Now, let's get the top 100 venues that are in The Beaches within a radius of 500 meters.¶

In [366]:
Limit=100
radius = 500 
url1 = "https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
    CLIENT_ID,CLIENT_SECRET,VERSION,first_to_neighborhood_latitude,first_to_neighborhood_longitude,radius,Limit)

url1

'https://api.foursquare.com/v2/venues/explore?&client_id=F5B4QDBCYPQ1ZMSJDVTAYEECCTMBRQNZTE3MQZOJ15WTEYBT&client_secret=W514NY3S2O2WC5DXF0AR5ABFIKI13T2XEZFRR4MLQSKMMTVH&v=20180605&ll=43.67635739999999,-79.2930312&radius=500&limit=100'

In [367]:
#get the results
import requests

results1 = requests.get(url1).json()
results1

{'meta': {'code': 200, 'requestId': '5c39c5671ed2193ebd8c11c4'},
 'response': {'headerLocation': 'The Beaches',
  'headerFullLocation': 'The Beaches, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 5,
  'suggestedBounds': {'ne': {'lat': 43.680857404499996,
    'lng': -79.28682091449052},
   'sw': {'lat': 43.67185739549999, 'lng': -79.29924148550948}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4b8daea1f964a520480833e3',
       'name': 'Grover Pub and Grub',
       'location': {'address': '676 Kingston Rd.',
        'crossStreet': 'at Main St.',
        'lat': 43.679181434941015,
        'lng': -79.29721535878515,
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.679181434941015,
          'lng': -79.29721535878515}],
    

In [368]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [369]:
venues1 = results1['response']['groups'][0]['items']
    
nearby_venues1 = json_normalize(venues1) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']

nearby_venues1 =nearby_venues1.loc[:, filtered_columns]


# filter the category for each row
nearby_venues1['venue.categories'] = nearby_venues1.apply(get_category_type, axis=1) #return each filter column name

print(nearby_venues.columns)

# clean columns
nearby_venues1.columns = [col.split('.')[-1] for col in nearby_venues1.columns]

nearby_venues1.head()

Index(['name', 'categories', 'lat', 'lng'], dtype='object')


,name,categories,lat,lng
0,Grover Pub and Grub,Pub,43.679181,-79.297215
1,Starbucks,Coffee Shop,43.678798,-79.298045
2,Upper Beaches,Neighborhood,43.680563,-79.292869
3,Beaches Fitness,Gym / Fitness Center,43.680319,-79.290991
4,Dip 'n Sip,Coffee Shop,43.678897,-79.297745


#### And how many venues near the beaches were returned by Foursquare

In [370]:
nearby_venues1.shape

(5, 4)

In [371]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

2 venues were returned by Foursquare.


### Let's create a function to repeat the same process to all the neighborhoods in Torronto

In [374]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]

    for name, lat, lng in zip(names, latitudes, longitudes):
        #print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID,
            CLIENT_SECRET,
            VERSION,
            lat, 
            lng, 
            radius, 
            Limit)
        
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        
        # return only relevant information for each nearby venue,here name,lat,lag are neighbourhood features
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])
        
    
    nearby_venues = pd.DataFrame([item for venue_list in venues_list
                                          for item in venue_list])
    
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighburhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
     
    return(nearby_venues)


### Get the venues

In [375]:
toronto_venues = getNearbyVenues(names=df_toronto['Neighborhood'],
                                   latitudes=df_toronto['Latitude'],
                                   longitudes=df_toronto['Longitude'])

### Let's check the size of the resulting dataframe

In [376]:
print(toronto_venues.shape)
toronto_venues.head()

(1707, 7)


,Neighborhood,Neighborhood Latitude,Neighburhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
1,The Beaches,43.676357,-79.293031,Starbucks,43.678798,-79.298045,Coffee Shop
2,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
3,The Beaches,43.676357,-79.293031,Beaches Fitness,43.680319,-79.290991,Gym / Fitness Center
4,The Beaches,43.676357,-79.293031,Dip 'n Sip,43.678897,-79.297745,Coffee Shop


In [377]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [378]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighburhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide, King, Richmond",100,100,100,100,100,100
Berczy Park,54,54,54,54,54,54
"Brockton, Exhibition Place, Parkdale Village",19,19,19,19,19,19
Business Reply Mail Processing Centre 969 Eastern,17,17,17,17,17,17
"CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara",14,14,14,14,14,14
"Cabbagetown, St. James Town",47,47,47,47,47,47
Central Bay Street,82,82,82,82,82,82
"Chinatown, Grange Park, Kensington Market",100,100,100,100,100,100
Christie,15,15,15,15,15,15


### Check about unique category

In [379]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 238 uniques categories.


## Analyze Each Neighborhood

In [380]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])

toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Arepa Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Store,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Piano Bar,Pizza Place,Plane,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,South American Restaurant,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,The Beaches,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [381]:
toronto_onehot.shape

(1707, 238)

### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category



In [382]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Arepa Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Store,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Piano Bar,Pizza Place,Plane,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,South American Restaurant,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store
0,"Adelaide, King, Richmond",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.040000,0.000000,0.00,0.00,0.010000,0.010000,0.000000,0.030000,0.000000,0.000000,0.000000,0.000000,0.030000,0.000000,0.030000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.020000,0.000000,0.01,0.020000,0.000000,0.000000,0.01,0.020000,0.010000,0.00,0.000000,0.050000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.030000,0.010000,0.060000,0.000000,0.000000,0.00,0.000000,0.000000,0.020000,0.000000,0.020000,0.000000,0.000000,0.0

In [383]:
toronto_grouped.shape

(38, 238)

### Let's print each neighborhood along with the top 5 most common venues

In [384]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide, King, Richmond----
                 venue  freq
0          Coffee Shop  0.06
1                 Café  0.05
2           Steakhouse  0.04
3      Thai Restaurant  0.04
4  American Restaurant  0.04


----Berczy Park----
                venue  freq
0         Coffee Shop  0.07
1          Restaurant  0.06
2        Cocktail Bar  0.06
3         Cheese Shop  0.04
4  Italian Restaurant  0.04


----Brockton, Exhibition Place, Parkdale Village----
            venue  freq
0  Breakfast Spot  0.11
1            Café  0.11
2     Coffee Shop  0.11
3    Climbing Gym  0.05
4             Bar  0.05


----Business Reply Mail Processing Centre 969 Eastern----
              venue  freq
0       Yoga Studio  0.06
1     Auto Workshop  0.06
2        Comic Shop  0.06
3              Park  0.06
4  Recording Studio  0.06


----CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara----
              venue  freq
0    Airport Lounge  0.14
1   Airport Service

### Function for sorting values

In [385]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

### create the new dataframe and display the top 10 venues for each neighborhood.



In [386]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
to_neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
to_neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
     to_neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)



### SO, here are the Ahmedabad Neighborhood with 10 venues for each neighborhood.

In [387]:
to_neighborhoods_venues_sorted.head(10)

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Café,Thai Restaurant,Steakhouse,American Restaurant,Hotel,Restaurant,Asian Restaurant,Bar,Gym
1,Berczy Park,Coffee Shop,Restaurant,Cocktail Bar,Cheese Shop,Bakery,Italian Restaurant,Beer Bar,Steakhouse,Seafood Restaurant,Pub
2,"Brockton, Exhibition Place, Parkdale Village",Coffee Shop,Café,Breakfast Spot,Bar,Stadium,Italian Restaurant,Burrito Place,Climbing Gym,Office,Furniture / Home Store
3,Business Reply Mail Processing Centre 969 Eastern,Yoga Studio,Auto Workshop,Park,Pizza Place,Recording Studio,Restaurant,Burrito Place,Brewery,Skate Park,Smoke Shop
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",Airport Lounge,Airport Terminal,Airport Service,Boutique,Sculpture Garden,Plane,Boat or Ferry,Airport Gate,Airport Food Court,Airport
5,"Cabbagetown, St. James Town",Coffee Shop,Restaurant,Italian Restaurant,Pizza Place,Market,Pub,Bakery,Café,Pet Store,Caribbean Restaurant
6,Central Bay Street,Coffee Shop,Café,Italian Restaurant,Bar,Burger Joint,Chinese Restaurant,Spa,Sandwich Place,Falafel Restaurant,Indian Restaurant
7,"Chinatown, Grange Park, Kensington Market",Bar,Café,Vegetarian / Vegan Restaurant,Dumpling Restaurant,Coffee Shop,Vietnamese Restaurant,Bakery,Chinese Restaurant,Mexican Restaurant,Caribbean Restaurant
8,Christie,Café,Grocery Store,Park,Convenience Store,Baby Store,Diner,Nightclub,Italian Restaurant,Restaurant,Coffee Shop
9,Church and Wellesley,Japanese Restaurant,Sushi Restaurant,Coffee Shop,Gay Bar,Restaurant,Burger Joint,Bubble Tea Shop,Café,Fast Food Restaurant,Men's Store


# Part-3 : Apply-Clustering on Neighborhood using k-means-clustering

In [ ]:
# set number of clusters
kclusters = 5


###  part-3: (1) Clustering Ahmedabad neighborhoods¶

In [389]:
ahmedabad_grouped.shape

(39, 85)

In [408]:
ahmedabad_clustering = ahmedabad_grouped.drop('Neighborhood', 1)

In [409]:
# run k-means clustering
kmeans_ah = KMeans(n_clusters=kclusters, random_state=0).fit(ahmedabad_clustering)

# check cluster labels generated for each row in the dataframe
kmeans_ah.labels_[0:10] 

array([2, 2, 3, 2, 2, 2, 2, 2, 2, 2], dtype=int32)

In [410]:
kmeans_ah.labels_.shape

(39,)

### Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [411]:
ahmedabad_merged_clusters = df_ah

# add clustering labels
ahmedabad_merged_clusters['Cluster Labels'] = kmeans_ah.labels_


In [412]:
# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
ahmedabad_merged_clusters = ahmedabad_merged_clusters.join(ah_neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

ahmedabad_merged_clusters.head() # check the last columns!

,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Asarwa,23.044980,72.607700,2,Business Service,Tea Room,Women's Store,Department Store,Coffee Shop,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop
1,Ashram Road,23.006986,72.557490,2,Indian Restaurant,Smoke Shop,Department Store,Coffee Shop,Flea Market,Pizza Place,Snack Place,Comfort Food Restaurant,Concert Hall,Construction & Landscaping
2,Astodia,23.016830,72.590975,3,Bus Station,Tea Room,Women's Store,Department Store,Coffee Shop,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop
3,Bapunagar,23.032660,72.629494,2,ATM,Arts & Crafts Store,Indian Restaurant,Movie Theater,Food,Food Court,Coffee Shop,Comfort Food Restaurant,Concert Hall,Construction & Landscaping
4,Behrampura,23.003589,72.583840,2,Wings Joint,Business Service,Baseball Field,Dessert Shop,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Cricket Ground


### part-3: (2) Clustering Toronto neighborhoods¶

In [421]:
# set number of clusters
kclusters = 5

toronto_clustering = toronto_grouped.drop('Neighborhood',1)

# run k-means clustering
kmeans_to = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_clustering)

# check cluster labels generated for each row in the dataframe
kmeans_to.labels_[0:10] 

array([2, 2, 2, 1, 2, 2, 2, 2, 2, 2], dtype=int32)

### Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [426]:
toronto_merged_clusters = df_toronto

# add clustering labels
toronto_merged_clusters['Cluster Labels'] = kmeans_to.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged_clusters = toronto_merged_clusters.join(to_neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged_clusters.head() # check the last columns!

,Postcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,2,Coffee Shop,Gym / Fitness Center,Pub,Women's Store,Discount Store,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,2,Greek Restaurant,Coffee Shop,Ice Cream Shop,Bookstore,Italian Restaurant,Yoga Studio,Cosmetics Shop,Brewery,Bubble Tea Shop,Restaurant
2,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572,2,Park,Sushi Restaurant,Board Shop,Brewery,Burger Joint,Sandwich Place,Burrito Place,Pub,Pet Store,Pizza Place
3,M4M,East Toronto,Studio District,43.659526,-79.340923,1,Café,Coffee Shop,Bakery,Italian Restaurant,American Restaurant,Yoga Studio,Coworking Space,Seafood Restaurant,Sandwich Place,Cheese Shop
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,2,Bus Line,Park,Swim School,Dim Sum Restaurant,Women's Store,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant


# Result : Examine and Visualize cluster

### Examine-Ahmedabad City Cluster

### Ahmedabad City-Cluster 1

In [428]:
ahmedabad_merged_clusters.loc[ahmedabad_merged_clusters['Cluster Labels'] == 0, \
                        ahmedabad_merged_clusters.columns[[0] + list(range(3, ahmedabad_merged_clusters.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
12,Delhi Chakla,0,Dessert Shop,Art Museum,Pizza Place,Ice Cream Shop,Women's Store,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop
13,Delhi Darwaja,0,Art Museum,Bus Stop,Pizza Place,Ice Cream Shop,Women's Store,Department Store,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store
19,Ghatlodia,0,Department Store,Pharmacy,Ice Cream Shop,Lake,Cricket Ground,Coffee Shop,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store
28,Shah Alam Road,0,Asian Restaurant,Lake,Ice Cream Shop,Women's Store,Dessert Shop,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop
30,Shahibaug Road,0,Track Stadium,Pizza Place,Convenience Store,Women's Store,Department Store,Coffee Shop,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Cosmetics Shop


### Ahmedabad City-Cluster 2

In [431]:
ahmedabad_merged_clusters.loc[ahmedabad_merged_clusters['Cluster Labels'] == 1, \
                        ahmedabad_merged_clusters.columns[[0] + list(range(3, ahmedabad_merged_clusters.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
26,Sarkej,1,Construction & Landscaping,Women's Store,Dessert Shop,Coffee Shop,Comfort Food Restaurant,Concert Hall,Convenience Store,Cosmetics Shop,Cricket Ground,Department Store


### Ahmedabad City-Cluster 3

In [432]:
ahmedabad_merged_clusters.loc[ahmedabad_merged_clusters['Cluster Labels'] == 2, \
                        ahmedabad_merged_clusters.columns[[0] + list(range(3, ahmedabad_merged_clusters.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Asarwa,2,Business Service,Tea Room,Women's Store,Department Store,Coffee Shop,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop
1,Ashram Road,2,Indian Restaurant,Smoke Shop,Department Store,Coffee Shop,Flea Market,Pizza Place,Snack Place,Comfort Food Restaurant,Concert Hall,Construction & Landscaping
3,Bapunagar,2,ATM,Arts & Crafts Store,Indian Restaurant,Movie Theater,Food,Food Court,Coffee Shop,Comfort Food Restaurant,Concert Hall,Construction & Landscaping
4,Behrampura,2,Wings Joint,Business Service,Baseball Field,Dessert Shop,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Cricket Ground
5,Bodakdev,2,Coffee Shop,Breakfast Spot,Café,Hotel,Tea Room,Indian Restaurant,Bubble Tea Shop,Women's Store,Concert Hall,Construction & Landscaping
6,Bopal,2,Mobile Phone Shop,Park,Fast Food Restaurant,Women's Store,Cricket Ground,Coffee Shop,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store
7,CG Road,2,Indian Restaurant,Pizza Place,Cricket Ground,Food,Fast Food Restaurant,Café,Ice Cream Shop,Clothing Store,Jewelry Store,Juice Bar
8,Chandkheda,2,Shopping Mall,Breakfast Spot,Indian Restaurant,Women's Store,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Cricket Ground
9,Chandlodia,2,Snack Place,Women's Store,Coffee Shop,Indian Restaurant,Dessert Shop,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop
11,Dariapur,2,Dessert Shop,Market,Bakery,Women's Store,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Cricket Ground


### Ahmedabad City-Cluster 4

In [433]:
ahmedabad_merged_clusters.loc[ahmedabad_merged_clusters['Cluster Labels'] == 3, \
                        ahmedabad_merged_clusters.columns[[0] + list(range(3, ahmedabad_merged_clusters.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Astodia,3,Bus Station,Tea Room,Women's Store,Department Store,Coffee Shop,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop
10,Dani Limbada,3,Bus Station,Lake,Women's Store,Department Store,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Cricket Ground
18,Geeta Mandir Road,3,Bus Station,Bistro,Tea Room,Dessert Shop,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Cricket Ground


### Ahmedabad City-Cluster 5

In [434]:
ahmedabad_merged_clusters.loc[ahmedabad_merged_clusters['Cluster Labels'] == 4, \
                        ahmedabad_merged_clusters.columns[[0] + list(range(3, ahmedabad_merged_clusters.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
29,Shahibagh,4,History Museum,Hotel,Women's Store,Dessert Shop,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Cricket Ground
33,Sola Road,4,History Museum,Hotel,Women's Store,Dessert Shop,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Cricket Ground


### Visualize - Ahmedabad City clusters

In [443]:
map_clusters_ahmedabad = folium.Map(location=[latitude, longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]

colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))

rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(ahmedabad_merged_clusters['Latitude'],\
        ahmedabad_merged_clusters['Longitude'], ahmedabad_merged_clusters['Neighborhood'], ahmedabad_merged_clusters['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters_ahmedabad)
 

In [445]:
map_clusters_ahmedabad

In [449]:
map_clusters_ahmedabad.save('ahmedabad_cluster.html')

### Examine-Toronto-City Clusters

### Toronto-City-Cluster 1

In [450]:
toronto_merged_clusters.loc[toronto_merged_clusters['Cluster Labels'] == 0, \
                            toronto_merged_clusters.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
24,"The Annex, North Midtown, Yorkville",0,Coffee Shop,Café,Sandwich Place,Pizza Place,History Museum,Indian Restaurant,Pharmacy,Cosmetics Shop,Pub,Burger Joint
27,"CN Tower, Bathurst Quay, Island airport, Harbo...",0,Airport Lounge,Airport Terminal,Airport Service,Boutique,Sculpture Garden,Plane,Boat or Ferry,Airport Gate,Airport Food Court,Airport


### Toronto-City-Cluster 2

In [451]:
toronto_merged_clusters.loc[toronto_merged_clusters['Cluster Labels'] == 1, \
                            toronto_merged_clusters.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,Studio District,1,Café,Coffee Shop,Bakery,Italian Restaurant,American Restaurant,Yoga Studio,Coworking Space,Seafood Restaurant,Sandwich Place,Cheese Shop
12,Church and Wellesley,1,Japanese Restaurant,Sushi Restaurant,Coffee Shop,Gay Bar,Restaurant,Burger Joint,Bubble Tea Shop,Café,Fast Food Restaurant,Men's Store
22,Roselawn,1,Garden,Pool,Music Venue,Women's Store,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant
36,"Runnymede, Swansea",1,Coffee Shop,Pizza Place,Café,Sushi Restaurant,Diner,Italian Restaurant,Gourmet Shop,Bookstore,Food,Bar


### Toronto-City- Cluster 3

In [452]:
toronto_merged_clusters.loc[toronto_merged_clusters['Cluster Labels'] == 2, \
                            toronto_merged_clusters.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,The Beaches,2,Coffee Shop,Gym / Fitness Center,Pub,Women's Store,Discount Store,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant
1,"The Danforth West, Riverdale",2,Greek Restaurant,Coffee Shop,Ice Cream Shop,Bookstore,Italian Restaurant,Yoga Studio,Cosmetics Shop,Brewery,Bubble Tea Shop,Restaurant
2,"The Beaches West, India Bazaar",2,Park,Sushi Restaurant,Board Shop,Brewery,Burger Joint,Sandwich Place,Burrito Place,Pub,Pet Store,Pizza Place
4,Lawrence Park,2,Bus Line,Park,Swim School,Dim Sum Restaurant,Women's Store,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant
5,Davisville North,2,Park,Hotel,Burger Joint,Food & Drink Shop,Dance Studio,Clothing Store,Sandwich Place,Breakfast Spot,Grocery Store,Gym
6,North Toronto West,2,Sporting Goods Shop,Coffee Shop,Clothing Store,Salon / Barbershop,Sandwich Place,Diner,Rental Car Location,Furniture / Home Store,Dessert Shop,Chinese Restaurant
7,Davisville,2,Pizza Place,Dessert Shop,Sandwich Place,Italian Restaurant,Seafood Restaurant,Café,Sushi Restaurant,Coffee Shop,Chinese Restaurant,Diner
8,"Moore Park, Summerhill East",2,Playground,Gym,Tennis Court,Park,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Diner,Electronics Store
9,"Deer Park, Forest Hill SE, Rathnelly, South Hi...",2,Pub,Coffee Shop,Light Rail Station,Supermarket,Sushi Restaurant,Bagel Shop,Sports Bar,American Restaurant,Pizza Place,Vietnamese Restaurant
10,Rosedale,2,Park,Playground,Trail,Diner,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store


###  Toronto-City-Cluster 4

In [453]:
toronto_merged_clusters.loc[toronto_merged_clusters['Cluster Labels'] == 3, \
                            toronto_merged_clusters.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
17,Central Bay Street,3,Coffee Shop,Café,Italian Restaurant,Bar,Burger Joint,Chinese Restaurant,Spa,Sandwich Place,Falafel Restaurant,Indian Restaurant


###  Toronto-City-Cluster 5

In [454]:
toronto_merged_clusters.loc[toronto_merged_clusters['Cluster Labels'] == 4, \
                            toronto_merged_clusters.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
28,Stn A PO Boxes 25 The Esplanade,4,Coffee Shop,Restaurant,Café,Italian Restaurant,Seafood Restaurant,Pub,Hotel,Beer Bar,Cocktail Bar,Japanese Restaurant


### Visualize - Toronto City clusters

In [458]:
map_clusters_toronto = folium.Map(location=[latitude1, longitude1], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]

colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))

rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged_clusters['Latitude'], toronto_merged_clusters['Longitude'],\
                                  toronto_merged_clusters['Neighborhood'], toronto_merged_clusters['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters_toronto)

In [459]:
map_clusters_toronto

In [460]:
map_clusters_toronto.save('toronto_cluster.html')

# Discusion:

# Thanks

In [462]:
ahmedabad_merged_clusters.head(10)

,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Asarwa,23.044980,72.607700,2,Business Service,Tea Room,Women's Store,Department Store,Coffee Shop,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop
1,Ashram Road,23.006986,72.557490,2,Indian Restaurant,Smoke Shop,Department Store,Coffee Shop,Flea Market,Pizza Place,Snack Place,Comfort Food Restaurant,Concert Hall,Construction & Landscaping
2,Astodia,23.016830,72.590975,3,Bus Station,Tea Room,Women's Store,Department Store,Coffee Shop,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop
3,Bapunagar,23.032660,72.629494,2,ATM,Arts & Crafts Store,Indian Restaurant,Movie Theater,Food,Food Court,Coffee Shop,Comfort Food Restaurant,Concert Hall,Construction & Landscaping
4,Behrampura,23.003589,72.583840,2,Wings Joint,Business Service,Baseball Field,Dessert Shop,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Cricket Ground
5,Bodakdev,23.044371,72.517921,2,Coffee Shop,Breakfast Spot,Café,Hotel,Tea Room,Indian Restaurant,Bubble Tea Shop,Women's Store,Concert Hall,Construction & Landscaping
6,Bopal,23.030050,72.464943,2,Mobile Phone Shop,Park,Fast Food Restaurant,Women's Store,Cricket Ground,Coffee Shop,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store
7,CG Road,23.039084,72.562786,2,Indian Restaurant,Pizza Place,Cricket Ground,Food,Fast Food Restaurant,Café,Ice Cream Shop,Clothing Store,Jewelry Store,Juice Bar
8,Chandkheda,23.112650,72.583618,2,Shopping Mall,Breakfast Spot,Indian Restaurant,Women's Store,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Cricket Ground
9,Chandlodia,23.093990,72.546181,2,Snack Place,Women's Store,Coffee Shop,Indian Restaurant,Dessert Shop,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop


In [463]:
toronto_merged_clusters.head(10)

,Postcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,2,Coffee Shop,Gym / Fitness Center,Pub,Women's Store,Discount Store,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,2,Greek Restaurant,Coffee Shop,Ice Cream Shop,Bookstore,Italian Restaurant,Yoga Studio,Cosmetics Shop,Brewery,Bubble Tea Shop,Restaurant
2,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572,2,Park,Sushi Restaurant,Board Shop,Brewery,Burger Joint,Sandwich Place,Burrito Place,Pub,Pet Store,Pizza Place
3,M4M,East Toronto,Studio District,43.659526,-79.340923,1,Café,Coffee Shop,Bakery,Italian Restaurant,American Restaurant,Yoga Studio,Coworking Space,Seafood Restaurant,Sandwich Place,Cheese Shop
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,2,Bus Line,Park,Swim School,Dim Sum Restaurant,Women's Store,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant
5,M4P,Central Toronto,Davisville North,43.712751,-79.390197,2,Park,Hotel,Burger Joint,Food & Drink Shop,Dance Studio,Clothing Store,Sandwich Place,Breakfast Spot,Grocery Store,Gym
6,M4R,Central Toronto,North Toronto West,43.715383,-79.405678,2,Sporting Goods Shop,Coffee Shop,Clothing Store,Salon / Barbershop,Sandwich Place,Diner,Rental Car Location,Furniture / Home Store,Dessert Shop,Chinese Restaurant
7,M4S,Central Toronto,Davisville,43.704324,-79.388790,2,Pizza Place,Dessert Shop,Sandwich Place,Italian Restaurant,Seafood Restaurant,Café,Sushi Restaurant,Coffee Shop,Chinese Restaurant,Diner
8,M4T,Central Toronto,"Moore Park, Summerhill East",43.689574,-79.383160,2,Playground,Gym,Tennis Court,Park,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Diner,Electronics Store
9,M4V,Central Toronto,"Deer Park, Forest Hill SE, Rathnelly, South Hi...",43.686412,-79.400049,2,Pub,Coffee Shop,Light Rail Station,Supermarket,Sushi Restaurant,Bagel Shop,Sports Bar,American Restaurant,Pizza Place,Vietnamese Restaurant


# Discussuion

### Discussion: For our business problem
1.	From cluster and   clustering map of Ahmedabad and Toronto, cluster-3 of Ahmedabad which also known as West Ahmedabad and cluster-3 of Toronto are seen like similar in form of Coffey-shop, pizza-Restaurant, Fast-food Restaurant, Café.

2.	For Tourism, As per the Ahmedabad cluster -1 it seems like Tourism place, also there heritage place Delhi Darwaza is situated. It’s also have lake in this cluster And the Ahmedabad cluster-3 seems like Residential Area.

3.	For Toronto no data shown for tourism in clusters, all clusters seems like residential area.

4.	For Optimal place for Business, Ahmedabad cluster-3 and Toronto cluster-3 are seems like good option for Business like Coffey-shop, fast-food restaurant, women’s store, boutique, Bookstore, pool, Hotel.

5.	From the both cluster of Ahmedabad and Toronto  we can say that in  Vegetarian ,Mexican, Fast-food ,pizaa and  Coffey shop are very famous.


6.	For Optimal location near Bus-station or Railway station,
a.	In Ahmedabad From the Cluster-4 ,we can say that Dani lambda and Astodia are best location. There are no data for Bus-station or Railway station in Toronto City.

7.
   > a.In Ahmedabad city, from the Cluster-2 Ahmedabad we can say  that Sardar nagar is near from the Airport and Best Option for Hotel Business, because very few hotel around them.

  >b.In Toronto City, From the Toronto-City-Cluster 1 we can say that CN Tower, Bathurst Quay are best for airport facility, and also airport longue is there.

# Conclusion

•	West Ahmedabad is little bit similar like Toronto.     
•	Ahmedabad have good heritage places and also have Residential Area, and all Toronto area are look like Residential Area.      
•	Good business option in Ahmedabad and Toronto are Coffey-shop, fast-food restaurant, women’s store, boutique, Bookstore, pool, Café.     
•	Vegetarian, Mexican, Fast-food, Pizza and Chinese food are famous in Ahmedabad and Canada.     
•	For Bus and Railway Station Dani lambda and Astodia are optimal location in Ahmedabad.     
•	For Airport Facility, CN Tower, Bathurst Quay in Toronto and Sardar Nagar in Ahmedabad are optimal place.   
 
•	We can say that, Both Toronto and Ahmedabad City are optimal place for Business.   


# Thanks For Reading.